In [56]:
import pandas as pd
import numpy as np

In [57]:
def load_and_prepare_data(file_path, sheet_name, start_date, end_date):  #works
    # Read the data source
    excel_file = pd.ExcelFile(file_path)
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    # Replace all NaN/- values with 0
    df = df.fillna(0)
    # Convert columns to datetime if necessary
    df.columns = [df.columns[0], df.columns[1]] + [pd.to_datetime(col) for col in df.columns[2:]]
    # Filter the DataFrame to include only the specified date range + work on a copy of the filtered df (data integrity)
    segmentation_period_df = df[['Item Code', 'Item Name'] + [col for col in df.columns[2:] if start_date <= col <= end_date]].copy()
    return segmentation_period_df


In [58]:
def abc_analysis(segmentation_period_df, a_threshold, b_threshold):  #works
    # Sum values row-wise excluding the first two columns
    row_sums = segmentation_period_df.iloc[:, 2:].sum(axis=1)
    segmentation_period_df['Sales Value'] = row_sums

    summarised_df = segmentation_period_df[['Item Code', 'Item Name', 'Sales Value']].copy()
    summarised_df.sort_values(by='Sales Value', ascending=False, inplace=True)

    total_revenue = summarised_df['Sales Value'].sum()
    num_items = len(summarised_df)

    summarised_df['Cumulative Sales Value'] = summarised_df['Sales Value'].cumsum()
    summarised_df['% Rev'] = (summarised_df['Cumulative Sales Value'] / total_revenue) * 100

    summarised_df['Rank'] = summarised_df.index + 1
    summarised_df['% Items'] = (summarised_df['Rank'] / num_items) * 100

    # Classify into categories A, B, and C
    abc_bins = [0, a_threshold, b_threshold, 100]
    abc_labels = ['A', 'B', 'C']
    summarised_df['ABC'] = pd.cut(summarised_df['% Rev'], bins=abc_bins, labels=abc_labels, right=True, include_lowest=True)
    return summarised_df


In [59]:
def abc_summary(summarised_df):  #works
    num_items = len(summarised_df)
    # Calculate the number of items in each category
    category_counts = summarised_df['ABC'].value_counts().sort_index()
    category_percentages = (category_counts / num_items) * 100

    summary = {
        "category_counts": category_counts.to_dict(),
        "category_percentages": category_percentages.to_dict()
    }
    return summary


In [60]:
def identify_outliers(segmentation_period_df, summarised_df):  #works
    # Calculate the sales standard deviation and mean
    sales_data_columns = segmentation_period_df.columns[2:-1]
    segmentation_period_df['Sales Std Dev'] = segmentation_period_df[sales_data_columns].std(axis=1)
    segmentation_period_df['Sales Mean'] = segmentation_period_df[sales_data_columns].mean(axis=1)
    
    # Merge these values into summarised_df
    summarised_df['Sales Std Dev'] = segmentation_period_df['Sales Std Dev']
    summarised_df['Sales Mean'] = segmentation_period_df['Sales Mean']

    # Calculate the coefficient of variation
    summarised_df['coefficient_variation_%'] = (summarised_df['Sales Std Dev'] / summarised_df['Sales Mean']) * 100

    # Check for NaN values in coefficient_variation_%
    nan_values = summarised_df[summarised_df['coefficient_variation_%'].isna()]

    # Handle NaN values: replace with a large number or some other logic
    summarised_df['coefficient_variation_%'].fillna(1000000, inplace=True)

    # Find values outside expected range
    negative_values = summarised_df[summarised_df['coefficient_variation_%'] < 0]
    excessive_values = summarised_df[summarised_df['coefficient_variation_%'] > 1000000]

    return {
        'nan_values': nan_values,
        'negative_values': negative_values,
        'excessive_values': excessive_values
    }




In [61]:
def xyz_analysis(summarised_df, x_threshold, y_threshold):

    # Ensure the coefficient_variation_% column exists
    if 'coefficient_variation_%' not in summarised_df.columns:
        raise KeyError("The 'coefficient_variation_%' column is missing in the DataFrame.")

    # Classify into categories X, Y, Z
    xyz_bins = [0, x_threshold, y_threshold, 1000000]
    xyz_labels = ['X', 'Y', 'Z']
    summarised_df['XYZ'] = pd.cut(summarised_df['coefficient_variation_%'], bins=xyz_bins, labels=xyz_labels, right=True, include_lowest=True)

    # Combine ABC and XYZ classifications
    summarised_df['ABCXYZ'] = summarised_df['ABC'].astype(str) + summarised_df['XYZ'].astype(str)

    # Define the custom sort order
    custom_sort_order = ['AX', 'AY', 'BX', 'AZ', 'BY', 'CX', 'BZ', 'CY', 'CZ']

    # Create a custom sort column based on the custom order
    summarised_df['sort_order'] = summarised_df['ABCXYZ'].apply(lambda x: custom_sort_order.index(x) if x in custom_sort_order else len(custom_sort_order))

    # Sort the DataFrame based on the custom sort order
    summarised_df = summarised_df.sort_values('sort_order').drop(columns=['sort_order'])

    return summarised_df


In [62]:
def abc_xyz_summary(summarised_df):  # Distribution summary of number of items in each category
    num_items = len(summarised_df)
    abcxyz_counts = summarised_df['ABCXYZ'].value_counts().sort_index()
    abcxyz_percentages = (abcxyz_counts / num_items) * 100
    summary = {
            "category_counts": abcxyz_counts.to_dict(),
            "category_percentages": abcxyz_percentages.to_dict()
        }
    return summary


In [63]:
# Execution
file_path = '/Users/arshiabansal/Desktop/ABC-Analysis-AbcSupplyChain.xlsx'
sheet_name = 'Unprocessed Data'
start_date = pd.to_datetime("2021-06-01")
end_date = pd.to_datetime("2022-06-01")

# Load and prepare data
segmentation_period_df = load_and_prepare_data(file_path, sheet_name, start_date, end_date)

# Perform ABC analysis
a_threshold = 20
b_threshold = 40
summarised_df = abc_analysis(segmentation_period_df, a_threshold, b_threshold)


In [64]:
summarised_df

,Item Code,Item Name,Sales Value,Cumulative Sales Value,% Rev,Rank,% Items,ABC
15,500067,item 16,13003299.0,13003299.0,3.308112,16,2.898551,A
0,500550,item 1,9198652.0,22201951.0,5.648301,1,0.181159,A
1,500061,item 2,8786040.0,30987991.0,7.883519,2,0.362319,A
3,500106,item 4,7327100.0,38315091.0,9.747574,4,0.724638,A
4,500551,item 5,7266600.0,45581691.0,11.596238,5,0.905797,A
...,...,...,...,...,...,...,...,...
443,500139,item 444,0.0,393073091.0,100.000000,444,80.434783,C
444,500140,item 445,0.0,393073091.0,100.000000,445,80.615942,C
445,500155,item 446,0.0,393073091.0,100.000000,446,80.797101,C
446,500157,item 447,0.0,393073091.0,100.000000,447,80.978261,C


In [65]:
# Identify outliers and add coefficient of variation
outliers = identify_outliers(segmentation_period_df, summarised_df)
print("NaN Values:", outliers['nan_values'])
print("Negative Values:", outliers['negative_values'])
print("Excessive Values:", outliers['excessive_values'])


NaN Values:      Item Code Item Name  Sales Value  Cumulative Sales Value  % Rev  Rank  \
496     500247  item 497          0.0             393073091.0  100.0   497   
492     500243  item 493          0.0             393073091.0  100.0   493   
495     500246  item 496          0.0             393073091.0  100.0   496   
494     500245  item 495          0.0             393073091.0  100.0   495   
493     500244  item 494          0.0             393073091.0  100.0   494   
..         ...       ...          ...                     ...    ...   ...   
443     500139  item 444          0.0             393073091.0  100.0   444   
444     500140  item 445          0.0             393073091.0  100.0   445   
445     500155  item 446          0.0             393073091.0  100.0   446   
446     500157  item 447          0.0             393073091.0  100.0   447   
551     500546  item 552          0.0             393073091.0  100.0   552   

        % Items ABC  Sales Std Dev  Sales Mean  coe

/var/folders/ns/1yxzfx892f36qy2_byrlq2kw0000gn/T/ipykernel_3212/2134158347.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  summarised_df['coefficient_variation_%'].fillna(1000000, inplace=True)


In [66]:
summarised_df

,Item Code,Item Name,Sales Value,Cumulative Sales Value,% Rev,Rank,% Items,ABC,Sales Std Dev,Sales Mean,coefficient_variation_%
15,500067,item 16,13003299.0,13003299.0,3.308112,16,2.898551,A,2.709057e+06,1.000254e+06,270.836921
0,500550,item 1,9198652.0,22201951.0,5.648301,1,0.181159,A,2.610071e+04,7.075886e+05,3.688684
1,500061,item 2,8786040.0,30987991.0,7.883519,2,0.362319,A,2.210599e+05,6.758492e+05,32.708462
3,500106,item 4,7327100.0,38315091.0,9.747574,4,0.724638,A,3.453195e+05,5.636231e+05,61.267800
4,500551,item 5,7266600.0,45581691.0,11.596238,5,0.905797,A,1.393711e+05,5.589692e+05,24.933583
...,...,...,...,...,...,...,...,...,...,...,...
443,500139,item 444,0.0,393073091.0,100.000000,444,80.434783,C,0.000000e+00,0.000000e+00,1000000.000000
444,500140,item 445,0.0,393073091.0,100.000000,445,80.615942,C,0.000000e+00,0.000000e+00,1000000.000000
445,500155,item 446,0.0,393073091.0,100.000000,446,80.797101,C,0.000000e+00,0.000000e+00,1000000.000000
446,500157,item 447,0.0,393073091.0,100.000000,447,80.978261,C,0.000000e+00,0.000000e+00,1000000.000000


In [67]:
# Perform XYZ analysis
x_threshold = 15
y_threshold = 35

completed_df = xyz_analysis(summarised_df, x_threshold, y_threshold)
print(completed_df)

     Item Code Item Name  Sales Value  Cumulative Sales Value       % Rev  \
0       500550    item 1    9198652.0              22201951.0    5.648301   
9       500069   item 10    4818600.0              75666991.0   19.250107   
1       500061    item 2    8786040.0              30987991.0    7.883519   
4       500551    item 5    7266600.0              45581691.0   11.596238   
5       500012    item 6    7200300.0              52781991.0   13.428035   
..         ...       ...          ...                     ...         ...   
219     500071  item 220     509800.0             370515691.0   94.261271   
214     500053  item 215     533200.0             370005891.0   94.131575   
222     500529  item 223     539700.0             369472691.0   93.995926   
251     500537  item 252     306400.0             383114891.0   97.466578   
551     500546  item 552          0.0             393073091.0  100.000000   

     Rank     % Items ABC  Sales Std Dev     Sales Mean  \
0       1    0.1

In [68]:
# Get ABC-XYZ summary
summary = abc_xyz_summary(completed_df)
print(summary)

{'category_counts': {'AX': 1, 'AY': 4, 'AZ': 5, 'BY': 9, 'BZ': 17, 'CX': 1, 'CY': 63, 'CZ': 452}, 'category_percentages': {'AX': 0.18115942028985507, 'AY': 0.7246376811594203, 'AZ': 0.9057971014492754, 'BY': 1.6304347826086956, 'BZ': 3.0797101449275366, 'CX': 0.18115942028985507, 'CY': 11.41304347826087, 'CZ': 81.88405797101449}}
